In [ ]:
## Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
import math
import scipy.stats as stats

In [ ]:
## Load files to the colab
dataset=pd.read_csv("sertaozinho.csv", delimiter=";")

## Add variable semester

In [ ]:
semester = []
for i in dataset['month']:
    if pd.isna(i):
        semester.append(None)
    elif i >= 7:
        semester.append(2)
    else:
        semester.append(1)
## semester = [2 if i >= 7 else 1 for i in dataset['month']]

In [ ]:
dataset.insert(5, 'semester', semester)

In [ ]:
dataset.head()

,id,grade_2012,grade_2017,follow_up,month,semester,year,school_2012,ethinicity,gender,failed_before_2012,mother_educ,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012
1,4769,4.0,NaN,0,NaN,NaN,NaN,municipal,NaN,mulher,NaN,NaN,NaN,NaN,NaN,NaN,-0.208333,-0.069444,0.395833,0.00,-1.000000,-1.263889,-0.791667,-0.525000,0.208333,-0.441667,NaN,NaN
2,5121,5.0,NaN,0,4.0,1.0,1999.0,municipal,preta,homem,sim,em,NaN,NaN,NaN,NaN,-0.420000,0.026667,-0.460000,-0.25,-0.216000,0.306667,-0.080000,0.048000,0.920000,1.216000,28.704331,0.873114
3,1586,5.0,10.0,1,2.0,1.0,2002.0,municipal,parda,homem,nao,ef2,nao,nao,sim,nao,-2.720000,0.426667,-1.360000,1.00,-0.256000,-0.426667,-1.280000,-0.032000,-1.280000,-1.344000,55.546427,52.508735
4,3230,5.0,9.0,1,5.0,1.0,2002.0,municipal,branca,homem,nao,ef2,nao,nao,nao,nao,0.200000,-1.266667,0.600000,-0.75,1.160000,0.933333,-0.200000,1.520000,-0.700000,1.240000,42.218278,42.168412
5,2410,5.0,10.0,1,7.0,2.0,2001.0,municipal,branca,homem,nao,nunca_estudou,nao,nao,sim,nao,0.152174,-2.347826,-1.423913,1.00,-1.130435,-0.217391,-0.152174,0.076087,-1.652174,-0.173913,63.801720,72.900984


In [ ]:
dataset.semester.value_counts(dropna=False)

1.0    2368
2.0    2045
NaN     511
Name: semester, dtype: int64

## Removendo linhas com follow_up 0 e retirando a coluna follow_up

In [ ]:
dataset = dataset[dataset.follow_up == 1]

dataset.follow_up.value_counts()

1    3151
Name: follow_up, dtype: int64

In [ ]:
dataset = dataset.drop(['follow_up'], axis = 1)

dataset.head()

,id,grade_2012,grade_2017,month,semester,year,school_2012,ethinicity,gender,failed_before_2012,mother_educ,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012
3,1586,5.0,10.0,2.0,1.0,2002.0,municipal,parda,homem,nao,ef2,nao,nao,sim,nao,-2.720000,0.426667,-1.360000,1.00,-0.256000,-0.426667,-1.280000,-0.032000,-1.280000,-1.344000,55.546427,52.508735
4,3230,5.0,9.0,5.0,1.0,2002.0,municipal,branca,homem,nao,ef2,nao,nao,nao,nao,0.200000,-1.266667,0.600000,-0.75,1.160000,0.933333,-0.200000,1.520000,-0.700000,1.240000,42.218278,42.168412
5,2410,5.0,10.0,7.0,2.0,2001.0,municipal,branca,homem,nao,nunca_estudou,nao,nao,sim,nao,0.152174,-2.347826,-1.423913,1.00,-1.130435,-0.217391,-0.152174,0.076087,-1.652174,-0.173913,63.801720,72.900984
6,2742,5.0,10.0,1.0,1.0,2002.0,municipal,parda,mulher,nao,nunca_estudou,sim,nao,nao,nao,0.120000,0.040000,-0.440000,0.00,-0.624000,0.626667,-2.120000,1.472000,-0.620000,-0.976000,61.105550,61.870584
10,1914,5.0,10.0,7.0,2.0,2001.0,municipal,parda,homem,nao,em,sim,nao,nao,nao,0.080000,-0.973333,-0.960000,1.00,0.184000,-0.360000,-1.080000,0.448000,-0.580000,0.016000,80.460214,77.081211


##Add Feature Atraso (dummie)

In [ ]:
print("Valores grade_2012\n", dataset.grade_2012.value_counts(dropna=False),"\n")
print("Valores grade_2017\n", dataset.grade_2017.value_counts(dropna=False),"\n")

Valores grade_2012
 5.0    1123
6.0    1016
4.0    1008
NaN       4
Name: grade_2012, dtype: int64 

Valores grade_2017
 10.0    1096
9.0      959
11.0     873
8.0      169
7.0       29
12.0      18
NaN        7
Name: grade_2017, dtype: int64 



In [ ]:
dataset = dataset[~dataset.grade_2012.isna()]
dataset = dataset[~dataset.grade_2017.isna()]

print(dataset.grade_2012.isna().sum(), dataset.grade_2017.isna().sum())

print(dataset.grade_2012.value_counts(dropna=False), dataset.grade_2017.value_counts(dropna=False))

0 0
5.0    1123
6.0    1016
4.0    1001
Name: grade_2012, dtype: int64 10.0    1094
9.0      958
11.0     872
8.0      169
7.0       29
12.0      18
Name: grade_2017, dtype: int64


In [ ]:
dataset['delay'] = (dataset['grade_2017'] - dataset['grade_2012'] < 5)
dataset['delay'] = dataset['delay'].astype('float')

In [ ]:
dataset.delay

3       0.0
4       1.0
5       0.0
6       0.0
10      0.0
       ... 
4904    0.0
4905    0.0
4906    0.0
4907    0.0
4915    0.0
Name: delay, Length: 3140, dtype: float64

##Delay Years (discreta)

In [ ]:
dataset["delay_years"] = 5 - (dataset['grade_2017'] - dataset['grade_2012'])

In [ ]:
dataset.head()

,id,grade_2012,grade_2017,month,semester,year,school_2012,ethinicity,gender,failed_before_2012,mother_educ,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years
3,1586,5.0,10.0,2.0,1.0,2002.0,municipal,parda,homem,nao,ef2,nao,nao,sim,nao,-2.720000,0.426667,-1.360000,1.00,-0.256000,-0.426667,-1.280000,-0.032000,-1.280000,-1.344000,55.546427,52.508735,0.0,0.0
4,3230,5.0,9.0,5.0,1.0,2002.0,municipal,branca,homem,nao,ef2,nao,nao,nao,nao,0.200000,-1.266667,0.600000,-0.75,1.160000,0.933333,-0.200000,1.520000,-0.700000,1.240000,42.218278,42.168412,1.0,1.0
5,2410,5.0,10.0,7.0,2.0,2001.0,municipal,branca,homem,nao,nunca_estudou,nao,nao,sim,nao,0.152174,-2.347826,-1.423913,1.00,-1.130435,-0.217391,-0.152174,0.076087,-1.652174,-0.173913,63.801720,72.900984,0.0,0.0
6,2742,5.0,10.0,1.0,1.0,2002.0,municipal,parda,mulher,nao,nunca_estudou,sim,nao,nao,nao,0.120000,0.040000,-0.440000,0.00,-0.624000,0.626667,-2.120000,1.472000,-0.620000,-0.976000,61.105550,61.870584,0.0,0.0
10,1914,5.0,10.0,7.0,2.0,2001.0,municipal,parda,homem,nao,em,sim,nao,nao,nao,0.080000,-0.973333,-0.960000,1.00,0.184000,-0.360000,-1.080000,0.448000,-0.580000,0.016000,80.460214,77.081211,0.0,0.0


In [ ]:
dataset.describe()

,id,grade_2012,grade_2017,month,semester,year,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years
count,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,2947.000000,2942.000000,2953.000000,2952.000000,2958.000000,2955.000000,2949.000000,2954.000000,2947.00000,2954.000000,2071.000000,2069.000000,3140.000000,3140.000000
mean,2664.666242,5.004777,9.848726,6.274204,1.464331,2001.199045,0.049391,-0.346971,-0.550496,0.226327,-0.073755,0.184444,-0.710248,0.249443,-0.50663,-0.139111,67.574707,60.145478,0.142994,0.156051
std,1526.387885,0.801585,0.938902,3.436537,0.498806,1.110435,0.927058,0.820865,0.656223,0.591800,0.632091,0.831201,0.814236,0.584051,0.85290,0.686910,18.793463,19.809214,0.350122,0.501522
min,5.000000,4.000000,7.000000,1.000000,1.000000,1994.000000,-3.200000,-3.181818,-2.833333,-2.015152,-2.400000,-2.053333,-3.120000,-2.750000,-2.56000,-2.571429,0.000000,0.032540,0.000000,-2.000000
25%,1357.250000,4.000000,9.000000,3.000000,1.000000,2000.000000,-0.560000,-0.893333,-1.020000,0.000000,-0.472000,-0.400000,-1.280000,-0.104000,-1.16000,-0.560000,59.209126,47.442352,0.000000,0.000000
50%,2695.000000,5.000000,10.000000,6.000000,1.000000,2001.000000,0.083333,-0.360000,-0.583333,0.250000,-0.088000,0.173333,-0.840000,0.256000,-0.64000,-0.136000,70.232553,61.587732,0.000000,0.000000
75%,3977.250000,6.000000,11.000000,9.000000,2.000000,2002.000000,0.750000,0.213333,-0.140000,0.554924,0.346870,0.773333,-0.260000,0.624000,0.02000,0.304000,81.087394,74.006902,0.000000,0.000000
max,5308.000000,6.000000,12.000000,12.000000,2.000000,2003.000000,2.458333,2.400000,2.120000,2.140351,2.048000,2.565217,3.680000,2.120000,3.04000,2.400000,100.000000,100.000000,1.000000,4.000000


## Tratamento dos dados para eles ficarem com média 0 e desvio padrão 1:

In [ ]:
dataset[['act_2012', 'aes_2012', 'alt_2012', 'anx_2012', 'ass_2012', 'cmp_2012', 'dep_2012', 'ids_2012',
       'ord_2012', 'sfd_2012', 'lang_2012', 'math_2012']]



,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012
3,-2.720000,0.426667,-1.360000,1.00,-0.256000,-0.426667,-1.280000,-0.032000,-1.280000,-1.344000,55.546427,52.508735
4,0.200000,-1.266667,0.600000,-0.75,1.160000,0.933333,-0.200000,1.520000,-0.700000,1.240000,42.218278,42.168412
5,0.152174,-2.347826,-1.423913,1.00,-1.130435,-0.217391,-0.152174,0.076087,-1.652174,-0.173913,63.801720,72.900984
6,0.120000,0.040000,-0.440000,0.00,-0.624000,0.626667,-2.120000,1.472000,-0.620000,-0.976000,61.105550,61.870584
10,0.080000,-0.973333,-0.960000,1.00,0.184000,-0.360000,-1.080000,0.448000,-0.580000,0.016000,80.460214,77.081211
...,...,...,...,...,...,...,...,...,...,...,...,...
4904,0.920000,-0.360000,-0.290000,-0.25,0.216000,-0.973333,0.080000,1.152000,0.580000,0.184000,80.984301,76.135556
4905,0.140000,-0.786667,-1.180000,0.50,-0.528000,-0.213333,-1.140000,-0.016000,-0.640000,-0.072000,85.065678,77.728851
4906,-0.369565,-0.956522,-1.184783,-0.50,-0.565217,0.000000,-0.130435,-0.321739,-1.130435,-0.373913,80.844563,65.013842
4907,0.920000,-1.026667,-1.540000,1.00,0.416000,0.360000,-0.920000,0.552000,-0.920000,-1.416000,62.294929,73.446855


In [ ]:
for i in ['act_2012', 'aes_2012', 'alt_2012', 'anx_2012', 'ass_2012', 'cmp_2012', 'dep_2012', 'ids_2012',
       'ord_2012', 'sfd_2012', 'lang_2012', 'math_2012']:
    dataset[i] = stats.zscore(dataset[i], nan_policy='omit')

In [ ]:
dataset.describe()

,id,grade_2012,grade_2017,month,semester,year,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years
count,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,3140.000000,2.947000e+03,2.942000e+03,2.953000e+03,2.952000e+03,2.958000e+03,2.955000e+03,2.949000e+03,2.954000e+03,2.947000e+03,2.954000e+03,2.071000e+03,2.069000e+03,3140.000000,3140.000000
mean,2664.666242,5.004777,9.848726,6.274204,1.464331,2001.199045,-1.424039e-17,6.037923e-18,-3.169380e-17,-1.161372e-16,3.347934e-17,-7.934995e-17,2.725675e-17,4.419845e-17,-6.961969e-17,-7.137148e-17,2.093663e-16,8.676803e-16,0.142994,0.156051
std,1526.387885,0.801585,0.938902,3.436537,0.498806,1.110435,1.000170e+00,1.000170e+00,1.000169e+00,1.000169e+00,1.000169e+00,1.000169e+00,1.000170e+00,1.000169e+00,1.000170e+00,1.000169e+00,1.000242e+00,1.000242e+00,0.350122,0.501522
min,5.000000,4.000000,7.000000,1.000000,1.000000,1994.000000,-3.505652e+00,-3.454074e+00,-3.479344e+00,-3.788201e+00,-3.680860e+00,-2.692675e+00,-2.960027e+00,-5.136457e+00,-2.407924e+00,-3.541553e+00,-3.596518e+00,-3.035328e+00,0.000000,-2.000000
25%,1357.250000,4.000000,9.000000,3.000000,1.000000,2000.000000,-6.574501e-01,-6.657061e-01,-7.155856e-01,-3.825023e-01,-6.301508e-01,-7.032502e-01,-6.998565e-01,-6.052608e-01,-7.661870e-01,-6.128311e-01,-4.452400e-01,-6.414286e-01,0.000000,0.000000
50%,2695.000000,5.000000,10.000000,6.000000,1.000000,2001.000000,3.661944e-02,-1.587454e-02,-5.004830e-02,4.000930e-02,-2.254065e-02,-1.336885e-02,-1.593810e-01,1.122848e-02,-1.563991e-01,4.530130e-03,1.414581e-01,7.282483e-02,0.000000,0.000000
75%,3977.250000,6.000000,11.000000,9.000000,2.000000,2002.000000,7.558625e-01,6.826944e-01,6.256499e-01,5.553454e-01,6.655613e-01,7.086000e-01,5.530639e-01,6.414175e-01,6.175624e-01,6.451880e-01,7.191836e-01,6.999155e-01,0.000000,0.000000
max,5308.000000,6.000000,12.000000,12.000000,2.000000,2003.000000,2.598923e+00,3.347004e+00,4.070186e+00,3.234790e+00,3.357291e+00,2.864741e+00,5.392776e+00,3.203273e+00,4.159023e+00,3.697049e+00,1.725766e+00,2.012405e+00,1.000000,4.000000


## Separando em dummies



Padrão da variável: dum_nome-da-variavel

Prefixo: nome-da-variavel

grade_2012

In [ ]:
dum_grade_2012 = pd.get_dummies(dataset['grade_2012'], prefix = 'grade_2012')
dum_grade_2012 = dum_grade_2012.drop(['grade_2012_4.0'], axis = 1)
dum_grade_2012.head()

,grade_2012_5.0,grade_2012_6.0
3,1,0
4,1,0
5,1,0
6,1,0
10,1,0


ethinicity

In [ ]:
dum_ethinicity = pd.get_dummies(dataset['ethinicity'], prefix = 'ethinicity')
dum_ethinicity = dum_ethinicity.drop(['ethinicity_amarela'], axis = 1)
dum_ethinicity.head()

,ethinicity_branca,ethinicity_indigena,ethinicity_parda,ethinicity_preta
3,0,0,1,0
4,1,0,0,0
5,1,0,0,0
6,0,0,1,0
10,0,0,1,0


gender

In [ ]:
dum_gender = pd.get_dummies(dataset['gender'], prefix = 'gender')
dum_gender = dum_gender.drop(['gender_homem'], axis = 1)
dum_gender.head()

,gender_mulher
3,0
4,0
5,0
6,1
10,0


mother_educ

In [ ]:
dum_mother_educ = pd.get_dummies(dataset['mother_educ'], prefix = 'mother_educ')
dum_mother_educ = dum_mother_educ.drop(['mother_educ_ef1'], axis = 1)
dum_mother_educ.head()

,mother_educ_ef2,mother_educ_em,mother_educ_nao_sabe,mother_educ_nunca_estudou,mother_educ_superior
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,0,0,0,1,0
10,0,1,0,0,0


school_2012

In [ ]:
dum_school_2012 = pd.get_dummies(dataset['school_2012'], prefix = 'school_2012')
dum_school_2012 = dum_school_2012.drop(['school_2012_estadual'], axis = 1)
dum_school_2012.head()

,school_2012_municipal,school_2012_privada
3,1,0
4,1,0
5,1,0
6,1,0
10,1,0


Adicionando as dummies e retirando as variaveis originais


In [ ]:
dataset = pd.concat([dataset, dum_ethinicity, dum_gender, dum_grade_2012,
                     dum_mother_educ, dum_school_2012], axis = 1)
dataset = dataset.drop(['ethinicity','gender','grade_2012','mother_educ','school_2012'], axis = 1)

dataset.head()

,id,grade_2017,month,semester,year,failed_before_2012,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years,ethinicity_branca,ethinicity_indigena,ethinicity_parda,ethinicity_preta,gender_mulher,grade_2012_5.0,grade_2012_6.0,mother_educ_ef2,mother_educ_em,mother_educ_nao_sabe,mother_educ_nunca_estudou,mother_educ_superior,school_2012_municipal,school_2012_privada
3,1586,10.0,2.0,1.0,2002.0,nao,nao,nao,sim,nao,-2.987797,0.942627,-1.233790,1.307544,-0.288370,-0.735338,-0.699856,-0.481963,-0.906907,-1.754367,-0.640179,-0.385608,0.0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
4,3230,9.0,5.0,1.0,2002.0,nao,nao,nao,nao,nao,0.162487,-1.120588,1.753507,-1.650037,1.952192,0.901125,0.626765,2.175791,-0.226759,2.008042,-1.349541,-0.907730,1.0,1.0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
5,2410,10.0,7.0,2.0,2001.0,nao,nao,nao,sim,nao,0.110889,-2.437910,-1.331202,1.307544,-1.672004,-0.483521,0.685512,-0.296867,-1.343344,-0.050673,-0.200809,0.644074,0.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
6,2742,10.0,1.0,1.0,2002.0,nao,sim,nao,nao,nao,0.076178,0.471499,0.168411,-0.382502,-0.870663,0.532119,-1.731673,2.093592,-0.132946,-1.218544,-0.344307,0.087107,0.0,0.0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
10,1914,10.0,7.0,2.0,2001.0,nao,sim,nao,nao,nao,0.033023,-0.763181,-0.624138,1.307544,0.407850,-0.655119,-0.454186,0.340023,-0.086039,0.225848,0.685803,0.855149,0.0,0.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


## Excluindo grade_2017

In [ ]:
dataset = dataset.drop(['grade_2017'], axis = 1)
dataset.head()

,id,month,semester,year,failed_before_2012,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years,ethinicity_branca,ethinicity_indigena,ethinicity_parda,ethinicity_preta,gender_mulher,grade_2012_5.0,grade_2012_6.0,mother_educ_ef2,mother_educ_em,mother_educ_nao_sabe,mother_educ_nunca_estudou,mother_educ_superior,school_2012_municipal,school_2012_privada
3,1586,2.0,1.0,2002.0,nao,nao,nao,sim,nao,-2.987797,0.942627,-1.233790,1.307544,-0.288370,-0.735338,-0.699856,-0.481963,-0.906907,-1.754367,-0.640179,-0.385608,0.0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
4,3230,5.0,1.0,2002.0,nao,nao,nao,nao,nao,0.162487,-1.120588,1.753507,-1.650037,1.952192,0.901125,0.626765,2.175791,-0.226759,2.008042,-1.349541,-0.907730,1.0,1.0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
5,2410,7.0,2.0,2001.0,nao,nao,nao,sim,nao,0.110889,-2.437910,-1.331202,1.307544,-1.672004,-0.483521,0.685512,-0.296867,-1.343344,-0.050673,-0.200809,0.644074,0.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
6,2742,1.0,1.0,2002.0,nao,sim,nao,nao,nao,0.076178,0.471499,0.168411,-0.382502,-0.870663,0.532119,-1.731673,2.093592,-0.132946,-1.218544,-0.344307,0.087107,0.0,0.0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
10,1914,7.0,2.0,2001.0,nao,sim,nao,nao,nao,0.033023,-0.763181,-0.624138,1.307544,0.407850,-0.655119,-0.454186,0.340023,-0.086039,0.225848,0.685803,0.855149,0.0,0.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


# Converting variables that had exclusively 'nao'/'sim' inputs into 0/1

It was checked whether missing value cells were affected due to this conversion(e.g. if data was introduced into an otherwise empty cell). Apparently, they weren't. 'nao' cells became 0, 'sim' cells became 1.

Function to check for missing value % before the conversion

In [ ]:
def percentage_of_missing_values(dataset):
    #creates a list with all columns within the dataset
    list_columns = dataset.columns
    missing = 0
    #iterates over each column,adding up the number of missing values in each column on the variable "missing"
    for column in list_columns:
        missing += dataset[column].isna().sum()
    #divides the quantity of total missing values with the total number of values.
    #Then the number is transformed into a percentage.
    return f"The percentage of missing values is {round((missing*100)/(dataset.shape[0]*dataset.shape[1]),2)}%." 

Percentage of missing values in the whole dataset before the conversion

In [ ]:
percentage_of_missing_values(dataset)

'The percentage of missing values is 3.47%.'

Dataset before the conversion. Notice the 'nao'/'sim' columns.

In [ ]:
dataset.head()

,id,month,semester,year,failed_before_2012,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years,ethinicity_branca,ethinicity_indigena,ethinicity_parda,ethinicity_preta,gender_mulher,grade_2012_5.0,grade_2012_6.0,mother_educ_ef2,mother_educ_em,mother_educ_nao_sabe,mother_educ_nunca_estudou,mother_educ_superior,school_2012_municipal,school_2012_privada
3,1586,2.0,1.0,2002.0,nao,nao,nao,sim,nao,-2.987797,0.942627,-1.233790,1.307544,-0.288370,-0.735338,-0.699856,-0.481963,-0.906907,-1.754367,-0.640179,-0.385608,0.0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
4,3230,5.0,1.0,2002.0,nao,nao,nao,nao,nao,0.162487,-1.120588,1.753507,-1.650037,1.952192,0.901125,0.626765,2.175791,-0.226759,2.008042,-1.349541,-0.907730,1.0,1.0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
5,2410,7.0,2.0,2001.0,nao,nao,nao,sim,nao,0.110889,-2.437910,-1.331202,1.307544,-1.672004,-0.483521,0.685512,-0.296867,-1.343344,-0.050673,-0.200809,0.644074,0.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
6,2742,1.0,1.0,2002.0,nao,sim,nao,nao,nao,0.076178,0.471499,0.168411,-0.382502,-0.870663,0.532119,-1.731673,2.093592,-0.132946,-1.218544,-0.344307,0.087107,0.0,0.0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
10,1914,7.0,2.0,2001.0,nao,sim,nao,nao,nao,0.033023,-0.763181,-0.624138,1.307544,0.407850,-0.655119,-0.454186,0.340023,-0.086039,0.225848,0.685803,0.855149,0.0,0.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


Conversion

In [ ]:
#list with column names that have 'nao'/'sim' cells
list_columns = ['failed_before_2012','pre_k_pub',	'pre_k_priv',	'kinder_pub',	'kinder_priv']
for column in list_columns:
    dataset[column].replace(('sim', 'nao'), (1, 0), inplace=True)

Dataset after conversion

In [ ]:
dataset.head()

,id,month,semester,year,failed_before_2012,pre_k_pub,pre_k_priv,kinder_pub,kinder_priv,act_2012,aes_2012,alt_2012,anx_2012,ass_2012,cmp_2012,dep_2012,ids_2012,ord_2012,sfd_2012,lang_2012,math_2012,delay,delay_years,ethinicity_branca,ethinicity_indigena,ethinicity_parda,ethinicity_preta,gender_mulher,grade_2012_5.0,grade_2012_6.0,mother_educ_ef2,mother_educ_em,mother_educ_nao_sabe,mother_educ_nunca_estudou,mother_educ_superior,school_2012_municipal,school_2012_privada
3,1586,2.0,1.0,2002.0,0,0,0,1,0,-2.987797,0.942627,-1.233790,1.307544,-0.288370,-0.735338,-0.699856,-0.481963,-0.906907,-1.754367,-0.640179,-0.385608,0.0,0.0,0,0,1,0,0,1,0,1,0,0,0,0,1,0
4,3230,5.0,1.0,2002.0,0,0,0,0,0,0.162487,-1.120588,1.753507,-1.650037,1.952192,0.901125,0.626765,2.175791,-0.226759,2.008042,-1.349541,-0.907730,1.0,1.0,1,0,0,0,0,1,0,1,0,0,0,0,1,0
5,2410,7.0,2.0,2001.0,0,0,0,1,0,0.110889,-2.437910,-1.331202,1.307544,-1.672004,-0.483521,0.685512,-0.296867,-1.343344,-0.050673,-0.200809,0.644074,0.0,0.0,1,0,0,0,0,1,0,0,0,0,1,0,1,0
6,2742,1.0,1.0,2002.0,0,1,0,0,0,0.076178,0.471499,0.168411,-0.382502,-0.870663,0.532119,-1.731673,2.093592,-0.132946,-1.218544,-0.344307,0.087107,0.0,0.0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
10,1914,7.0,2.0,2001.0,0,1,0,0,0,0.033023,-0.763181,-0.624138,1.307544,0.407850,-0.655119,-0.454186,0.340023,-0.086039,0.225848,0.685803,0.855149,0.0,0.0,0,0,1,0,0,1,0,0,1,0,0,0,1,0


Notice how missing values % of the entire dataset didn't change.

In [ ]:
percentage_of_missing_values(dataset)

'The percentage of missing values is 3.47%.'

## Droping NAs


In [ ]:
dataset = dataset.dropna()

#Gerando arquivo que será usado para treinar os modelos

In [ ]:
dataset.to_csv('sertaozinhoTIDY.csv', header = True, index = False)